In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv


In [2]:
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torch import optim
from torch import nn
import torch
from torch import amp

In [3]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [4]:
class MyDataset2(Dataset):
    def __init__(self, text_list, tokenizer, max_len):
        self.text_list = text_list
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.text_list)
    
    def __getitem__(self, index):
        text = self.text_list[index]
        tokenized = self.tokenizer(text=text,padding='max_length',max_length=self.max_len,truncation=True,return_tensors='pt')
        return tokenized['input_ids'].squeeze(), tokenized['attention_mask'].squeeze()

In [5]:
class MyDataset(Dataset):
    def __init__(self, text_list, tokenizer, max_len, label_list):
        self.text_list = text_list
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_list = label_list
    
    def __len__(self):
        return len(self.text_list)
    
    def __getitem__(self, index):
        text = self.text_list[index]
        label = self.label_list[index]
        tokenized = self.tokenizer(text=text,padding='max_length',max_length=self.max_len,truncation=True,return_tensors='pt')
        return tokenized['input_ids'].squeeze(), tokenized['attention_mask'].squeeze(), label

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
torch.cuda.empty_cache()

In [8]:
model_path = 'microsoft/deberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)
model.to(device)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/475 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

DebertaModel(
  (embeddings): DebertaEmbeddings(
    (word_embeddings): Embedding(50265, 1024, padding_idx=0)
    (LayerNorm): DebertaLayerNorm()
    (dropout): StableDropout()
  )
  (encoder): DebertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x DebertaLayer(
        (attention): DebertaAttention(
          (self): DisentangledSelfAttention(
            (in_proj): Linear(in_features=1024, out_features=3072, bias=False)
            (pos_dropout): StableDropout()
            (pos_proj): Linear(in_features=1024, out_features=1024, bias=False)
            (pos_q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): StableDropout()
          )
          (output): DebertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): DebertaLayerNorm()
            (dropout): StableDropout()
          )
        )
        (intermediate): DebertaIntermediate(
          (dense): Linear(in_features=1024, out_f

In [9]:
# hyperparameters
max_len = 768
batch_size = 8
num_epoch = 1
learning_rage = 0.00001

In [10]:
train_df = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
train_texts = train_df['text']
train_labels = train_df['generated']
train_dataset = MyDataset(train_texts, tokenizer, max_len, train_labels)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size,num_workers=2,pin_memory=False ) ## 并发

In [11]:
optimizer = optim.Adam(model.parameters(), lr=learning_rage)
loss_fn = nn.BCEWithLogitsLoss()

In [12]:
for ep in range(num_epoch):
    model.train()
    total_loss = 0
    for j, (batch_input_ids, batch_attention_mask, batch_labels) in enumerate(train_dataloader):
        batch_input_ids = batch_input_ids.to(device)
        batch_attention_mask = batch_attention_mask.to(device)
        batch_labels = batch_labels.float().to(device)
        
        with amp.autocast(device_type='cuda'):
            output = model(batch_input_ids, batch_attention_mask)
        loss = loss_fn(output.view(-1), batch_labels)
        total_loss = total_loss + loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'epoch: {ep}, loss: {total_loss/len(train_dataloader)}')    
    
torch.save(model.state_dict(), 'final_epoch.pth')    
    

OutOfMemoryError: CUDA out of memory. Tried to allocate 288.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 190.12 MiB is free. Process 3526 has 14.55 GiB memory in use. Of the allocated memory 14.38 GiB is allocated by PyTorch, and 46.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
predict_model = AutoModel.from_pretrained(model_path)
predict_model.load_state_dict(torch.load('final_epoch.pth'))
predict_model.to(device)

In [ ]:
test_df = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
test_texts = test_df['text']
test_dataset = MyDataset2(test_texts, tokenizer, max_len)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False )

In [ ]:
predict_model.eval()
pred_prob = []
for (batch_input_ids, batch_attention_mask) in test_loader:
    batch_input_ids = batch_input_ids.to(device)
    batch_attention_mask = batch_attention_mask.to(device)
    logits = predict_model(batch_input_ids, batch_attention_mask)
    pred_prob.append(logits.sigmod().to('cpu').squeeze())

In [ ]:
id_list = test_df['id']
submission = pd.DataFrame(data={'id': id_list, 'generated': pred_prob})
submission.to_csv('submission.csv', index=False)
submission.head()